In [1]:
import bs4
import os
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv


load_dotenv()  # take environment variables from .env

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["USER_AGENT"] = os.getenv("USER_AGENT")

llm = ChatOpenAI(model="gpt-4o-mini")


### Simple Translate

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

llm.invoke(messages)


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

result = prompt_template.invoke({"language": "Italian", "text": "hi!"})

print(result)
print(result.to_messages())


In [ ]:
chain = prompt_template | llm

response = chain.invoke({"language": "Italian", "text": "hi!"})
print(response.content)


## Loading

In [5]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())


In [ ]:
print(len(docs[0].page_content))
print(docs[0].page_content[:500])


In [ ]:
# print(len(splits))
# print(len(splits[0].page_content))
print("\n".join([
    f"\n\nSplit {i+1} \n------------------\n{splits[i].page_content}"
    for i in range(min(12, len(splits)))
]))


## Inference

### Prompt

In [ ]:
# Retrieve and generate using the relevant snippets of the blog.
prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

print(example_messages[0].content)


#### Alternative RAG Inference

In [ ]:
similiarity_retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = similiarity_retriever.invoke("What are the approaches to Task Decomposition?")

print(retrieved_docs[0].page_content)


### Main Chain Inference

In [ ]:

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retriever = vectorstore.as_retriever()

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("What is Task Decomposition?"):
    print(chunk, end="", flush=True)


## Cleanup

In [11]:

# cleanup
vectorstore.delete_collection()
